In [1]:
!nvidia-smi

Wed Dec  6 02:22:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Get the data from the kaggle, create a directory to temporarily store the data and upzip the data.

In [2]:
import json

# To access kaggle, providing my credentials
kaggle_api_key = {
    'username': 'tsantosh1098',
    'key': '844743c89e375b419c3b2c6c08e4efd8'
}

# Save the Kaggle API key information to a file (temporary)
kaggle_json_path = './kaggle.json'
with open(kaggle_json_path, 'w') as f:
    json.dump(kaggle_api_key, f)

In [3]:
# Create the .kaggle directory
!mkdir -p ~/.kaggle

# Move the Kaggle JSON file to the .kaggle directory
!mv kaggle.json ~/.kaggle/

# Set appropriate permissions for the Kaggle JSON file
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
# Download the data from the kaggle dataset
!kaggle competitions download -c predict-ai-model-runtime

100% 2.71G/2.71G [01:33<00:00, 37.1MB/s]
100% 2.71G/2.71G [01:33<00:00, 31.2MB/s]


In [5]:
# Upzip the data
!unzip predict-ai-model-runtime.zip -d predict-ai-model-runtime

Streaming output truncated to the last 5000 lines.
  inflating: predict-ai-model-runtime/npz_all/npz/tile/xla/train/mlperf_nmt_batch_64_4636a292ae989fc5.npz  
  inflating: predict-ai-model-runtime/npz_all/npz/tile/xla/train/mlperf_nmt_batch_64_47566a605d8af12f.npz  
  inflating: predict-ai-model-runtime/npz_all/npz/tile/xla/train/mlperf_nmt_batch_64_47fa67ae30e8a55b.npz  
  inflating: predict-ai-model-runtime/npz_all/npz/tile/xla/train/mlperf_nmt_batch_64_48f4a5c651574b3c.npz  
  inflating: predict-ai-model-runtime/npz_all/npz/tile/xla/train/mlperf_nmt_batch_64_496d520844a6a5b9.npz  
  inflating: predict-ai-model-runtime/npz_all/npz/tile/xla/train/mlperf_nmt_batch_64_4a344dac5561baf2.npz  
  inflating: predict-ai-model-runtime/npz_all/npz/tile/xla/train/mlperf_nmt_batch_64_4a77e25058ba8a1c.npz  
  inflating: predict-ai-model-runtime/npz_all/npz/tile/xla/train/mlperf_nmt_batch_64_4b5dcc7c2341434d.npz  
  inflating: predict-ai-model-runtime/npz_all/npz/tile/xla/train/mlperf_nmt_batch_64_

### Some of the required packages installation

In [6]:
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 18.0 MB/s eta 0:00:00


In [7]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.3 MB/s eta 0:00:00


### Import necessary packages.

In [8]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor

import numpy as np
import pandas as pd
import os
from tqdm import tqdm

import sklearn,sklearn.model_selection
import torch
from torch import nn
from torch import Tensor
from torch_geometric.nn import GCNConv, SAGEConv, MLP
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from torch.utils.data import Dataset
from torch.optim.lr_scheduler import OneCycleLR
import torchmetrics as tm

import matplotlib.pyplot as plt


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
torch.backends.cudnn.deterministic = True

### Class to handle Tile data

In [9]:
class TileDataset(Dataset):

    def __init__(self, directory: str, split: str = "test"):
        """
        Initialize the TileDataset.

        Args:
        - directory (str): The directory containing the dataset.
        - split (str): The name of the dataset directory to be choosen (default is "test").
        """
        self.file_path_list = self.__get_path(directory, split)
        self.device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
        self.load_all_data()

    def __get_path(self, directory: str, split: str = "test"):
        """
        Get the file paths for the specified split.

        Args:
        - directory (str): The directory containing the dataset.
        - split (str): The split of the dataset (default is "test").

        Returns:
        - list: List of file paths for the specified split.
        """
        path = os.path.join(directory, split)
        files = os.listdir(path)
        list_path = [os.path.join(path, file) for file in files]
        return list_path

    def __len__(self):
        """
        Get the total number of samples in the dataset.

        Returns:
        - int: The number of samples in the dataset.
        """
        return len(self.file_path_list)


    def __tile(self, file):
        """
        Process a file to extract features and target.

        Args:
        - file: Dictionary containing data from a file.

        Returns:
        - tuple: Tuple containing torch tensors for config_feat and target.
        """
        config_feat = torch.tensor(file.get("config_feat").astype(np.float32)) * 100
        target = file['config_runtime'].astype(np.float32) / file['config_runtime_normalizers'].astype(np.float32)
        return config_feat, target


    def load_all_data(self):
        """
        Load all data from the dataset and preprocess it.
        """
        self.all_data = []
        for file in tqdm(self.file_path_list):
            file = dict(np.load(file))
            node_feat = torch.tensor(file['node_feat'].astype(np.float32))
            node_opcode = torch.tensor(file['node_opcode'].astype(np.int64))
            edge_index = torch.tensor(np.swapaxes(file['edge_index'],0,1).astype(np.int64))
            config_feat, target_raw = self.__tile(file)

            # minmax scale the target values
            target = (target_raw - (min(target_raw)-1.0e-7))/(max(target_raw) -(min(target_raw)-1.0e-7))
            target = torch.tensor(target)

            self.all_data.append(Data(
                node_feat = node_feat,
                node_opcode = node_opcode,
                edge_index = edge_index,
                config_feat = config_feat,
                target = target,
                target_raw = torch.tensor(target_raw)).to(device))


    def __getitem__(self, idx):
        """
        Get a sample from the dataset.

        Args:
        - idx (int): Index of the sample.

        Returns:
        - torch_geometric.data.Data: The data sample at the specified index.
        """
        return self.all_data[idx]

### Evaluation Metric

In [10]:
class EvaluationMetric(tm.Metric):
    higher_is_better = True

    def __init__(self, k:int=5) -> None:
        super().__init__()
        self.add_state("runtimes", default=[], dist_reduce_fx=None)
        self.k = k

    def update(self, preds: torch.Tensor, target: torch.Tensor) -> None:
        """
        Update the metric state
        Args:
            preds: Tensor of shape (bs, seq_len) with the predicted runtimes orders
            target: Tensor of shape (bs, seq_len) with the target runtimes
        """
        best_runtimes = target.min()
        best_predicted_runtimes = target[preds.argsort()[:self.k]].min()
        self.runtimes.append(best_predicted_runtimes/ best_runtimes)

    def compute(self) -> torch.Tensor:
        return (2 - torch.tensor(self.runtimes)).mean()

### Loss Function

In [11]:
# Adopted the loss function from here, https://github.com/allegro/allRank
def listMLE(y_pred, y_true, eps=1e-10, padded_value_indicator=-1):
    """
    ListMLE loss introduced in "Listwise Approach to Learning to Rank - Theory and Algorithm".
        y_pred: predictions from the model, shape [batch_size, slate_length]
        y_true: ground truth labels, shape [batch_size, slate_length]
        eps: epsilon value, used for numerical stability
        padded_value_indicator: an indicator of the y_true index containing a padded item, e.g. -1

    return: loss value, a torch.Tensor
    """

    # shuffle for randomised tie resolution
    random_indices = torch.randperm(y_pred.shape[-1])
    y_pred_shuffled = y_pred[random_indices]
    y_true_shuffled = y_true[random_indices]

    y_true_sorted, indices = y_true_shuffled.sort(descending = True, dim=-1)
    mask = y_true_sorted == padded_value_indicator

    preds_sorted_by_true = torch.gather(y_pred_shuffled, dim=0, index=indices)
    preds_sorted_by_true[mask] = float("-inf")

    max_pred_values, _ = preds_sorted_by_true.max(dim=0, keepdim=True)

    preds_sorted_by_true_minus_max = preds_sorted_by_true - max_pred_values
    cumsums = torch.cumsum(preds_sorted_by_true_minus_max.exp().flip(dims=[0]), dim=0).flip(dims=[0])
    observation_loss = torch.log(cumsums + eps) - preds_sorted_by_true_minus_max
    observation_loss[mask] = 0.0
    return torch.mean(torch.sum(observation_loss, dim=0))


### PyTorch Lightning to Train, Test and Validate the model using data efficiently.

PyTorch Lightning serves as a deep learning framework tailored for professional AI researchers and machine learning engineers, offering unparalleled flexibility without compromising scalability. As projects progress from initial ideas to research papers or production environments, Lightning seamlessly evolves to meet the evolving demands of our work.

In [14]:
class LightningWrapper(pl.LightningModule):
    def __init__(self, model:nn.Module):
        """
        LightningWrapper constructor.

        Args:
        - model (nn.Module): The PyTorch model to be wrapped.
        """
        super().__init__()
        self.model = model
        self.val_eval = EvaluationMetric()
        self.train_eval = EvaluationMetric()
        self.loss_function = listMLE
        self.start = False

    def forward(self, x):
        """
        Forward pass through the wrapped model.

        Args:
        - x: Input data.

        Returns:
        - Tensor: Model output.
        """
        return self.model(x)

    def training_step(self, batch):
        """
        Training step.

        Args:
        - batch: Batch of training data.

        Returns:
        - Tensor: Training loss.
        """
        outputs = self.model(batch)
        loss = self.loss_function(outputs, batch.target)
        self.train_eval.update(outputs, batch.target_raw)
        return loss

    def validation_step(self, batch):
        """
        Validation step.

        Args:
        - batch: Batch of validation data.

        Returns:
        - Tensor: Validation loss.
        """
        outputs = self.model(batch)
        loss = self.loss_function(outputs, batch.target)
        self.log("val_loss", loss, prog_bar=True)
        if not self.start:
            self.log("val_accuracy", 0)
            self.start = True
        self.val_eval.update(outputs, batch.target_raw)
        return loss

    def on_validation_epoch_end(self) -> None:
        """
        Called at the end of each validation epoch.
        Computes and logs evaluation metrics.
        """
        topk = self.val_eval.compute()
        topk_train = self.train_eval.compute()

        self.log("train_accuracy", topk_train, prog_bar=True)
        self.train_eval.reset()

        self.log("val_accuracy", topk, prog_bar=True)
        self.val_eval.reset()
        return super().on_validation_epoch_end()

    def predict_step(self, batch):
        """
        Prediction step.

        Args:
        - batch: Batch of data for prediction.

        Returns:
        - Tensor: Model predictions.
        """
        return self.model(batch)

    def configure_optimizers(self):
        """
        Configure optimizers and learning rate scheduler.

        Returns:
        - dict: Optimizer and LR scheduler configuration.
        """
        optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=0.001,
        )

        lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=4, min_lr=0.000001)

        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": lr_scheduler,
                "interval": "epoch",
                "frequency": 1,
                "monitor": "val_accuracy",
                "name": "LR"
            },
        }
        return optimizer

### Model to learn

In [67]:
class TileModel(torch.nn.Module):
    def __init__(self, embedding_dim, hidden_channels, graph_feats, hidden_dim, list_dense_layers_uints):
        super().__init__()
        self.op_embedding_dim = embedding_dim
        self.node_feature_lenght = 140
        self.op_code = 120
        self.hidden_channels = hidden_channels
        self.graph_feats = graph_feats
        self.hidden_dim = hidden_dim
        self.list_dense_layers_uints = list_dense_layers_uints

        self.embedding = torch.nn.Embedding(
            self.op_code,
            self.op_embedding_dim,
        )

        assert len(self.hidden_channels)>0
        self.convs = torch.nn.ModuleList()


        in_channels = self.op_embedding_dim + self.node_feature_lenght
        last_dim = self.hidden_channels[0]
        self.convs.append(GCNConv(in_channels, self.hidden_channels[0]))

        for i in range(len(self.hidden_channels)-1):
            self.convs.append(GCNConv(self.hidden_channels[i], self.hidden_channels[i+1]))
            last_dim = self.hidden_channels[i+1]
        self.convs.append(GCNConv(last_dim, graph_feats))

        self.dense = MLP(channel_list=list_dense_layers_uints, dropout=0.2, norm=None)



    def forward(self, data: Data) -> Tensor:

        x = torch.concat(
            [
                data.node_feat,
                self.embedding(data.node_opcode)
            ],
            dim = 1
        )

        #pass though conv layers
        for conv in self.convs:
            x = conv(x, data.edge_index).relu()

        # get 1d graph embedding using average pooling
        x_graph = torch.mean(x,0)

        #put graph data into config data
        x = torch.concat([data.config_feat, x_graph.repeat((len(data.config_feat),1))],axis=1) #torch.Size([10528, 225])

        x = torch.flatten(self.dense(x))

        x = (x - x.mean()) / (x.std() + 1e-8)

        return x

### Use the custom dataloader - TileDataset and get train, test and validation data.

In [16]:
train_dataset_tile_xla = TileDataset("./predict-ai-model-runtime/npz_all/npz/tile/xla/", split = "train")
validation_dataset_tile_xla = TileDataset("./predict-ai-model-runtime/npz_all/npz/tile/xla/", split = "valid")
test_dataset_tile_xla = TileDataset("./predict-ai-model-runtime/npz_all/npz/tile/xla/", split = "test")

100%|██████████| 844/844 [00:03<00:00, 277.45it/s]


### Write a checkpoint function, training configuration and instantiate a model and pass the model to PyTorch Lightning.

In [ ]:
NUM_EPOCH = 10

checkpoint_callback = ModelCheckpoint(
    monitor='val_accuracy',
    dirpath='tile_xla',
    filename='./model-{epoch:02d}-val_acc-{val_accuracy:.3f}',
    auto_insert_metric_name=False,
    save_on_train_epoch_end = False,  # run checkpointing after validation
    save_top_k = 5,
    mode = 'max',
    verbose = True,
)

lr_monitor = LearningRateMonitor(logging_interval='epoch')

trainer_config = dict(
    max_epochs= NUM_EPOCH,
    precision= 32,
    accumulate_grad_batches = 8,
    check_val_every_n_epoch = 1,
    callbacks = [checkpoint_callback, lr_monitor]
)

### 1. Training a GNN model with encoding op codes with a 128-dimensional embedding, utilizing graph hidden layers with units [32, 64, 64, 32], setting graph features to be 128, and employing a 5-layer MLP for further processing.

In [29]:
# Hyperparameters

p_embedding_dim   = 128
p_hidden_channels = [32,64,64,32]
p_graph_feats     = 128
p_hidden_dim      = 128
p_list_dense_layers_uints = [152, 64, 32, 16, 1]


model = TileModel(embedding_dim = p_embedding_dim,
                  hidden_channels = p_hidden_channels,
                  graph_feats = p_graph_feats,
                  hidden_dim = p_hidden_dim,
                  list_dense_layers_uints = p_list_dense_layers_uints).to(device)
model = LightningWrapper(model)

In [36]:
trainer = pl.Trainer(**trainer_config,)
trainer.fit(model, train_dataset_tile_xla, validation_dataset_tile_xla)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type             | Params
------------------------------------------------
0 | model      | TileModel        | 49.0 K
1 | val_eval   | EvaluationMetric | 0     
2 | train_eval | EvaluationMetric | 0     
------------------------------------------------
49.0 K    Trainable params
0         Non-trainable params
49.0 K    Total params
0.196     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 714: 'val_accuracy' reached 0.93774 (best 0.93774), saving model to '/content/tile_xla/./model-00-val_acc-0.938.ckpt' as top 5


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 7, global step 5712: 'val_accuracy' was not in top 5


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 8, global step 6426: 'val_accuracy' reached 0.93739 (best 0.93774), saving model to '/content/tile_xla/./model-08-val_acc-0.937.ckpt' as top 5


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 9, global step 7140: 'val_accuracy' was not in top 5
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


### 2. Training a GNN model with encoding op codes with a 64-dimensional embedding, utilizing graph hidden layers with units [32, 64, 64, 32], setting graph features to be 128, and employing a 5-layer MLP for further processing.

In [38]:
# Hyperparameters

p_embedding_dim   = 64
p_hidden_channels = [32,64,64,32]
p_graph_feats     = 128
p_hidden_dim      = 128
p_list_dense_layers_uints = [152, 64, 32, 16, 1]


model = TileModel(embedding_dim = p_embedding_dim,
                  hidden_channels = p_hidden_channels,
                  graph_feats = p_graph_feats,
                  hidden_dim = p_hidden_dim,
                  list_dense_layers_uints = p_list_dense_layers_uints).to(device)
model = LightningWrapper(model)

In [39]:
trainer = pl.Trainer(**trainer_config,)
trainer.fit(model, train_dataset_tile_xla, validation_dataset_tile_xla)

INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory /content/tile_xla exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type             | Params
------------------------------------------------
0 | model      | TileModel        | 39.2 K
1 | val_eval   | EvaluationMetric

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: The ``compute`` method of metric EvaluationMetric was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 7, global step 5712: 'val_accuracy' was not in top 5


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 8, global step 6426: 'val_accuracy' was not in top 5


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 9, global step 7140: 'val_accuracy' reached 0.94447 (best 0.94447), saving model to '/content/tile_xla/./model-09-val_acc-0.944.ckpt' as top 5
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


### 3. Training a GNN model with encoding op codes with a 64-dimensional embedding, utilizing graph hidden layers with units [32, 64, 64, 32], setting graph features to be 64, and employing a 4-layer MLP for further processing.

In [50]:
# Hyperparameters

p_embedding_dim   = 64
p_hidden_channels = [32,64,64,32]
p_graph_feats     = 64
p_hidden_dim      = 64
p_list_dense_layers_uints = [88, 64, 16, 1]


model = TileModel(embedding_dim = p_embedding_dim,
                  hidden_channels = p_hidden_channels,
                  graph_feats = p_graph_feats,
                  hidden_dim = p_hidden_dim,
                  list_dense_layers_uints = p_list_dense_layers_uints).to(device)
model = LightningWrapper(model)

In [51]:
trainer = pl.Trainer(**trainer_config,)
trainer.fit(model, train_dataset_tile_xla, validation_dataset_tile_xla)

INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type             | Params
------------------------------------------------
0 | model      | TileModel        | 31.5 K
1 | val_eval   | EvaluationMetric | 0     
2 | train_eval | EvaluationMetric | 0     
------------------------------------------------
31.5 K    Trainable params
0         Non-trainable para

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 714: 'val_accuracy' was not in top 5


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 7, global step 5712: 'val_accuracy' was not in top 5


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 8, global step 6426: 'val_accuracy' reached 0.94510 (best 0.94510), saving model to '/content/tile_xla/./model-08-val_acc-0.945.ckpt' as top 5


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 9, global step 7140: 'val_accuracy' was not in top 5
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


### 4. Training a GNN - SAGE model with encoding op codes with a 128-dimensional embedding, utilizing graph hidden layers with units [32, 64, 64, 32], setting graph features to be 128, and employing a 5-layer MLP for further processing.

In [69]:
class TileModel(torch.nn.Module):
    def __init__(self, embedding_dim, hidden_channels, graph_feats, hidden_dim, list_dense_layers_uints):
        super().__init__()
        self.op_embedding_dim = embedding_dim
        self.node_feature_lenght = 140
        self.op_code = 120
        self.hidden_channels = hidden_channels
        self.graph_feats = graph_feats
        self.hidden_dim = hidden_dim
        self.list_dense_layers_uints = list_dense_layers_uints

        self.embedding = torch.nn.Embedding(
            self.op_code,
            self.op_embedding_dim,
        )

        assert len(self.hidden_channels)>0
        self.convs = torch.nn.ModuleList()


        in_channels = self.op_embedding_dim + self.node_feature_lenght
        last_dim = self.hidden_channels[0]
        self.convs.append(SAGEConv(in_channels, self.hidden_channels[0]))

        for i in range(len(self.hidden_channels)-1):
            self.convs.append(SAGEConv(self.hidden_channels[i], self.hidden_channels[i+1]))
            last_dim = self.hidden_channels[i+1]
        self.convs.append(SAGEConv(last_dim, graph_feats))

        self.dense = MLP(channel_list=list_dense_layers_uints, dropout=0.2, norm=None)



    def forward(self, data: Data) -> Tensor:

        x = torch.concat(
            [
                data.node_feat,
                self.embedding(data.node_opcode)
            ],
            dim = 1
        )

        #pass though conv layers
        for conv in self.convs:
            x = conv(x, data.edge_index).relu()

        # get 1d graph embedding using average pooling
        x_graph = torch.mean(x,0)

        #put graph data into config data
        x = torch.concat([data.config_feat, x_graph.repeat((len(data.config_feat),1))],axis=1) #torch.Size([10528, 225])

        x = torch.flatten(self.dense(x))

        x = (x - x.mean()) / (x.std() + 1e-8)

        return x

In [56]:
# Hyperparameters

p_embedding_dim   = 128
p_hidden_channels = [32,64,64,32]
p_graph_feats     = 128
p_hidden_dim      = 128
p_list_dense_layers_uints = [152, 64, 32, 16, 1]


model = TileModel(embedding_dim = p_embedding_dim,
                  hidden_channels = p_hidden_channels,
                  graph_feats = p_graph_feats,
                  hidden_dim = p_hidden_dim,
                  list_dense_layers_uints = p_list_dense_layers_uints).to(device)
model = LightningWrapper(model)

In [57]:
trainer = pl.Trainer(**trainer_config,)
trainer.fit(model, train_dataset_tile_xla, validation_dataset_tile_xla)

INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type             | Params
------------------------------------------------
0 | model      | TileModel        | 69.8 K
1 | val_eval   | EvaluationMetric | 0     
2 | train_eval | EvaluationMetric | 0     
------------------------------------------------
69.8 K    Trainable params
0         Non-trainable para

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 714: 'val_accuracy' was not in top 5


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 7, global step 5712: 'val_accuracy' reached 0.94236 (best 0.94510), saving model to '/content/tile_xla/./model-07-val_acc-0.942.ckpt' as top 5


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 8, global step 6426: 'val_accuracy' reached 0.94448 (best 0.94510), saving model to '/content/tile_xla/./model-08-val_acc-0.944.ckpt' as top 5


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 9, global step 7140: 'val_accuracy' was not in top 5
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


### 5. Training a GNN - SAGE model with encoding op codes with a 64-dimensional embedding, utilizing graph hidden layers with units [32, 64, 64, 32], setting graph features to be 128, and employing a 5-layer MLP for further processing.

In [58]:
# Hyperparameters

p_embedding_dim   = 64
p_hidden_channels = [32,64,64,32]
p_graph_feats     = 128
p_hidden_dim      = 128
p_list_dense_layers_uints = [152, 64, 32, 16, 1]


model = TileModel(embedding_dim = p_embedding_dim,
                  hidden_channels = p_hidden_channels,
                  graph_feats = p_graph_feats,
                  hidden_dim = p_hidden_dim,
                  list_dense_layers_uints = p_list_dense_layers_uints).to(device)
model = LightningWrapper(model)

In [59]:
trainer = pl.Trainer(**trainer_config,)
trainer.fit(model, train_dataset_tile_xla, validation_dataset_tile_xla)

INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type             | Params
------------------------------------------------
0 | model      | TileModel        | 58.0 K
1 | val_eval   | EvaluationMetric | 0     
2 | train_eval | EvaluationMetric | 0     
------------------------------------------------
58.0 K    Trainable params
0         Non-trainable para

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 714: 'val_accuracy' was not in top 5


...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 7, global step 5712: 'val_accuracy' was not in top 5


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 9, global step 7140: 'val_accuracy' reached 0.94579 (best 0.94579), saving model to '/content/tile_xla/./model-09-val_acc-0.946.ckpt' as top 5
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


### 6. Training a GNN - SAGE model with encoding op codes with a 64-dimensional embedding, utilizing graph hidden layers with units [32, 64, 64, 32], setting graph features to be 64, and employing a 4-layer MLP for further processing.

In [60]:
# Hyperparameters

p_embedding_dim   = 64
p_hidden_channels = [32,64,64,32]
p_graph_feats     = 64
p_hidden_dim      = 64
p_list_dense_layers_uints = [88, 64, 16, 1]


model = TileModel(embedding_dim = p_embedding_dim,
                  hidden_channels = p_hidden_channels,
                  graph_feats = p_graph_feats,
                  hidden_dim = p_hidden_dim,
                  list_dense_layers_uints = p_list_dense_layers_uints).to(device)
model = LightningWrapper(model)

In [61]:
trainer = pl.Trainer(**trainer_config,)
trainer.fit(model, train_dataset_tile_xla, validation_dataset_tile_xla)

INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type             | Params
------------------------------------------------
0 | model      | TileModel        | 48.2 K
1 | val_eval   | EvaluationMetric | 0     
2 | train_eval | EvaluationMetric | 0     
------------------------------------------------
48.2 K    Trainable params
0         Non-trainable para

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 714: 'val_accuracy' was not in top 5


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 7, global step 5712: 'val_accuracy' was not in top 5


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 8, global step 6426: 'val_accuracy' was not in top 5


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [64]:
!zip -r lightning_logs.zip /content/lightning_logs

  adding: content/lightning_logs/ (stored 0%)
  adding: content/lightning_logs/version_4/ (stored 0%)
  adding: content/lightning_logs/version_4/events.out.tfevents.1701830465.7ef10923945d.1613.4 (deflated 59%)
  adding: content/lightning_logs/version_4/hparams.yaml (stored 0%)
  adding: content/lightning_logs/version_11/ (stored 0%)
  adding: content/lightning_logs/version_11/events.out.tfevents.1701832457.7ef10923945d.1613.11 (deflated 60%)
  adding: content/lightning_logs/version_11/hparams.yaml (stored 0%)
  adding: content/lightning_logs/version_14/ (stored 0%)
  adding: content/lightning_logs/version_14/events.out.tfevents.1701833754.7ef10923945d.1613.14 (deflated 59%)
  adding: content/lightning_logs/version_14/hparams.yaml (stored 0%)
  adding: content/lightning_logs/version_12/ (stored 0%)
  adding: content/lightning_logs/version_12/events.out.tfevents.1701833076.7ef10923945d.1613.12 (deflated 12%)
  adding: content/lightning_logs/version_12/hparams.yaml (stored 0%)
  adding: 

### Finally we were able to get the validation Accuracy of 94.6 %.

### The following section utilizes the trained model to run predictions on test data and extract the top 5 tiles for the specified neural subgraph.

In [81]:
submission_df = pd.read_csv('./predict-ai-model-runtime/sample_submission.csv')


best_model_path = "/content/tile_xla/model-09-val_acc-0.946.ckpt"


import torch
import torch_geometric
from torch_geometric.data import Data

# Create an instance of your model
p_embedding_dim   = 64
p_hidden_channels = [32,64,64,32]
p_graph_feats     = 128
p_hidden_dim      = 128
p_list_dense_layers_uints = [152, 64, 32, 16, 1]


model = TileModel(embedding_dim = p_embedding_dim,
                  hidden_channels = p_hidden_channels,
                  graph_feats = p_graph_feats,
                  hidden_dim = p_hidden_dim,
                  list_dense_layers_uints = p_list_dense_layers_uints).to(device)
model = LightningWrapper(model)

# Load the trained weights from the checkpoint
checkpoint_path = "/content/tile_xla/model-09-val_acc-0.946.ckpt"
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])

list_tile_xla_predictions = []
for data in test_dataset_tile_xla:
    tile_xla_predictions = model.predict_step(data)
    list_tile_xla_predictions.append(tile_xla_predictions)

tile_xla_predictions = [torch.argsort(pred) for pred in list_tile_xla_predictions]

In [82]:
len(tile_xla_predictions)

844

In [87]:
sub = submission_df
for idx, filename in enumerate(test_dataset_tile_xla.file_path_list):
    id = 'tile:xla:' + filename[:-4].split("/")[-1]
    sub.loc[sub.ID == id,'TopConfigs'] = ';'.join(tile_xla_predictions[idx][0:5].cpu().numpy().astype(str))

In [88]:
sub.to_csv('submission.csv',index=False)
sub

,ID,TopConfigs
0,tile:xla:d6f5f54247bd1e58a10b9e7062c636ab,7;5;21;2;22
1,tile:xla:e3a655daa38e34ec240df959b650ac16,118;30;99;431;1005
2,tile:xla:f8c2c1a1098b2a361c26df668b286c87,116;2;91;41;129
3,tile:xla:4dd1716853ed46ee4e7d09ede1732de8,5985;5146;8087;5487;9292
4,tile:xla:d0a69155b6340748c36724e4bfc34be3,153;159;849;41;171
...,...,...
889,layout:nlp:random:60880ed76de53f4d7a1b960b24f2...,0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
890,layout:nlp:random:23559853d9702baaaacbb0c83fd3...,0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
891,layout:nlp:random:f6c146fc5cf10be4f3accbaca989...,0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
892,layout:nlp:random:32531d07a084b319dce484f53a4c...,0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...


Colab link to our project file: https://colab.research.google.com/drive/1pyKKrHpwHX8r_GoHMaT3FdX7LscEj1_q?usp=sharing